# Downloading and coverting ERA5 files

Information on the fifth generation ECMWF atmospheric reanalysis of the global climate ERA5 is found at:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

# 1) Requesting ERA5 data

a) Go to the ERA5 hourly data on single levels from 1979 to present (https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form)

b) In product type select Reanalysis

c) In Variable select Popular -> 10m u-component of wind, 10m v-component of wind, and Mean sea level pressure.

d) In this example we will prepare the data for Hurricane Arthur, from 06/27/2014 to 07/09/2014. Therefore, we will have to download 2 ERA5 files, one from 06/27/2014 to 06/30/2014 and another from 07/01/2014 to 07/09/2014. 

First, we will download the data from 06/27/2014 to 06/30/2014. To do that we just need to select the Year, Month and Day accordingly. We will work with inputs every 6 hours, so select the Time: 00:00, 06:00, 12:00, and 18:00.

e) In the Geographical area tab select “Whole available region”

f) Select the GRIB format in the Format tab.

g) Click on “Show API request” and save the code.


# 2) Downloading ERA5 data

Importing the necessary lybraries:


cfgrib is installed as:
conda install -c conda-forge cfgrib
xarray is installed as:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [1]:
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import cdsapi
c = cdsapi.Client()

Paste the API request and start downloading:

In [ ]:
time = ['00:00', '01:00', '02:00','03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00','09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
days = ['17','18','19','20','21','22','23','24','25','26','27','28','29','30']
#['10','11','22','23','24','25','26','27','28','29','30']#,
        #
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure',
        ],
        'year': '2015',
        'month': '09',
        'day': days,
        'time': time,
    },
    'download.grib')

2020-05-05 07:40:03,041 INFO Welcome to the CDS
2020-05-05 07:40:03,042 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-05-05 07:40:03,773 INFO Request is completed
2020-05-05 07:40:03,774 INFO Downloading http://136.156.132.153/cache-compute-0002/cache/data9/adaptor.mars.internal-1588637084.5329049-20545-3-d93417be-7814-4455-9b2d-cc387e1804c2.grib to download.grib (1.8G)
 52%|█████▏    | 956M/1.81G [41:57<20:51, 752kB/s]     

# 3) Converting the .grib file into .22 (NWS=6)

Set the path to where the .grib data was saved:

Open the .grib file:

In [2]:
path = pl.Path(r'D:\FHRL_work\libraries\adcirc-unswan\inputs')
file = 'download.grib'
grib = xr.open_dataset(path / file, engine='cfgrib')

2020-04-29 14:26:20,935 INFO ecCodes library found using name 'eccodes'.
2020-04-29 14:26:20,944 WARNING Ignoring index file 'D:\\FHRL_work\\libraries\\adcirc-unswan\\inputs\\download.grib.4cc40.idx' older than GRIB file
2020-04-29 14:26:29,443 INFO missing from GRIB stream: 'directionNumber'
2020-04-29 14:26:29,444 INFO missing from GRIB stream: 'frequencyNumber'
2020-04-29 14:26:29,784 INFO missing from GRIB stream: 'directionNumber'
2020-04-29 14:26:29,785 INFO missing from GRIB stream: 'frequencyNumber'
2020-04-29 14:26:30,085 INFO missing from GRIB stream: 'directionNumber'
2020-04-29 14:26:30,086 INFO missing from GRIB stream: 'frequencyNumber'


Extract the variables of interest:

In [5]:
u = grib.data_vars['u10'].values[:]
v = grib.data_vars['v10'].values[:]
prmsl = grib.data_vars['msl'].values[:]
lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
#time = grib.coords['time'].values[:]

Create boundaries that cover the extent of your mesh (in our case the North Atlantic and the Caribean):

In [3]:
lat1,lat2 = 30,90
lon1,lon2 = -220, -115

Create a grid in using the format expected for a NWS = 6 in 3 columns (uwind, vwind, pressure):

In [13]:
temp,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []

Start populating the grid with data from the .grib file:

In [14]:
for t in range(0,temp):
    for i in range(0,y):
        for ii in range(0,x):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(prmsl[t,i,ii])
                datay.append(i)
                datax.append(ii)
    print('time = '+str(t))


time = 0
time = 1
time = 2
time = 3
time = 4
time = 5
time = 6
time = 7
time = 8
time = 9
time = 10
time = 11
time = 12
time = 13
time = 14
time = 15
time = 16
time = 17
time = 18
time = 19
time = 20
time = 21
time = 22
time = 23
time = 24
time = 25
time = 26
time = 27
time = 28
time = 29
time = 30
time = 31
time = 32
time = 33
time = 34
time = 35
time = 36
time = 37
time = 38
time = 39
time = 40
time = 41
time = 42
time = 43
time = 44
time = 45
time = 46
time = 47
time = 48
time = 49
time = 50
time = 51
time = 52
time = 53
time = 54
time = 55
time = 56
time = 57
time = 58
time = 59
time = 60
time = 61
time = 62
time = 63
time = 64
time = 65
time = 66
time = 67
time = 68
time = 69
time = 70
time = 71
time = 72
time = 73
time = 74
time = 75
time = 76
time = 77
time = 78
time = 79
time = 80
time = 81
time = 82
time = 83
time = 84
time = 85
time = 86
time = 87
time = 88
time = 89
time = 90
time = 91
time = 92
time = 93
time = 94
time = 95
time = 96
time = 97
time = 98
time = 99
time = 100

In [15]:
df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('fort.22', sep='\t',index=False,header = False)  

Repeat steps 1, 2, and 3 for the second part of the storm

Make sure that when you repeat the process for the other part of the storm (07/01/2014 to 07/09/2014) you change the name of the output file.

Use a text editor (Sublime for instance) to copy the 'arthur_2.22' file at the end of the 'arthur_1.22' file. This complete file refers to the whole storm.

Check the extent of your grid:

In [7]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(240, 420)

# Updating the fort.15 file:

Change the line YYYY MM DD HH24 to:

212 168 60.0 -100.0 0.25 0.25 21600 3600 ! YYYY MM DD HH24 StormNumber BLAdj 

where, 212 is the number of values in Y, 168 is the number of values in X, 60 is the maximum Y, -100 is the minimum X, 0.25 are the X and Y increments, and 21600 is the time interval in seconds (6hours).